# Mixed-integer programming

Suppose Joey wants to carry items to the pawn shop to get some extra cash. He has $N$ items, each with a weight $w_i$ and a price $p_i$. Iain hasn't been to the gym lately, so he can only carry $C$ kilos. How does he choose what to bring with him?

We can model this as an integer optimization problem:

\begin{align*}
\max& \sum_{i=1}^N p_i x_i \\
\text{s.t.}& \sum_{i=1}^N w_i x_i \leq C \\
& x_i \in \{0,1\} \quad \forall i = 1,\ldots,N
\end{align*}


# The knapsack problem

Consider the following (small) knapsack problem:

\begin{align*}
    \max\:& x + y \\
    \text{s.t.}\:& x + 2y \leq 1.5 \\
    & x, y \in \{0,1\}
\end{align*}

How would you solve this? 

## Implementation in JuMP

Let's solve our simple knapsack problem and see what the solver spits out.

In [ ]:
using JuMP, Cbc
m = Model(solver=CbcSolver(logLevel=1))

@variable(m, x, Bin)
@variable(m, y, Bin)

@constraint(m, x + 2y ≤ 1.5)
@objective(m, Max, x + y)

solve(m)

This is kind of dull since Cbc solves this before we ever get to the branch-and-bound tree! Let's cook up a problem that's a little more interesting. What about more items, and more knapsacks! If $N=100$, naïve enumeration would create $2^{100}$ nodes, which would take quite some time. How does the solver actually tackle it?

In [ ]:
srand(100)
N = 100

m = Model(solver=CbcSolver(logLevel=1))
@variable(m, x[1:N], Bin)
for _ in 1:10
    @constraint(m, dot(rand(N), x) ≤ N / 50)
end

@objective(m, Max, dot(rand(N), x))

solve(m)

## Sudoku

![Sudoku](http://upload.wikimedia.org/wikipedia/commons/f/ff/Sudoku-by-L2G-20050714.svg)

**Sudoku** is a number puzzle played on a 9x9 grid. The challenge is to place a digit between 1 and 9 inclusive in each empty cell, such that the completed grid obeys the following rules:

* Each row contains the numbers 1 to 9 once and only once.
* Each column contains the numbers 1 to 9 once and only once.
* Each 3x3 subgrid contains the numbers 1 to 9 once and only once.

In [ ]:
init_vals = [
5 3 0 0 7 0 0 0 0
6 0 0 1 9 5 0 0 0
0 9 8 0 0 0 0 6 0
8 0 0 0 6 0 0 0 3
4 0 0 8 0 3 0 0 1
7 0 0 0 2 0 0 0 6
0 6 0 0 0 0 2 8 0
0 0 0 4 1 9 0 0 5
0 0 0 0 8 0 0 7 9
]

The most natural formulation of this problem would probably be something like

$$x_{i,j} \in \{1, 2, \dots, 9\}$$

which is of course something we can do with integer programming:

$$1 \leq x_{i,j} \leq 9, \quad x_{i,j} \in \mathbb{Z}$$

The challenge now is the constraints. One observation is that the numbers 1 to 9 add up to 45, so we could try something like:

$$ \sum_{i=1}^9 x_{i,j} = 45 \quad \forall j$$
$$ \sum_{j=1}^9 x_{i,j} = 45 \quad \forall i$$
$$ \sum_{i=k}^{k+2} \sum_{j=\ell}^{\ell+2} x_{i,j} = 45 \quad \forall k,\ell \in \{1,3,7\} $$

In [ ]:
sudoku = Model(solver=CbcSolver(logLevel=0))
@variable(sudoku, 1 ≤ x[1:9,1:9] ≤ 9, Int)
    
@constraints(sudoku, begin
    rows[i=1:9], sum(x[i,:]) == 45
    cols[j=1:9], sum(x[:,j]) == 45
    subgrid[i=1:3:7,j=1:3:7], sum(x[i:i+2,j:j+2]) == 45
end)
    
for i in 1:9, j in 1:9
    if init_vals[i,j] != 0
        @constraint(sudoku, x[i,j] == init_vals[i,j])
    end
end
    
solve(sudoku)
    
getvalue(x)

Well that didn't work. Could we make this work at all?

Instead, let's change our **variables**: $x_{i,j,k} = 1$ iff the number $k$ will appear in cell $(i,j)$. We can now use our 0-1 integer programming knowledge to model the problem. Consider the "row" constraints: we want each number to appear once and only once. This is equivalent to saying that

$$\sum_{j=1}^9 x_{i,j,k} = 1 \quad \forall i, k$$

We can now model this as a $9\times 9\times 9$ dimensional problem - thats a lot of binary variables, surely Gurobi won't like that!

In [ ]:
sudoku = Model(solver=CbcSolver(logLevel=1))

@variable(sudoku, x[i=1:9, j=1:9, k=1:9], Bin)

@constraints(sudoku, begin
    # Constraint 1 - Exactly one value appears in each cell.
    cell[i=1:9, j=1:9], sum(x[i,j,:]) == 1
    # Constraint 2 - Each value appears in each row exactly once.
    row[i=1:9, k=1:9], sum(x[i,:,k]) == 1
    # Constraint 3 - Each value appears in each column exactly once.
    col[j=1:9, k=1:9], sum(x[:,j,k]) == 1
    # Constraint 4 - Each value appears in each 3x3 subgrid exactly once.
    subgrid[i=1:3:7,j=1:3:7,val=1:9], sum(x[i:i+2,j:j+2,val]) == 1
end)

# Fix given values. 
for row in 1:9, col in 1:9
    if init_vals[row,col] != 0
        @constraint(sudoku, x[row, col, init_vals[row, col]] == 1)
    end
end
    
solve(sudoku)
    
soln = getvalue(x)

In [ ]:
# soln2 is a 9x9x9 array with values 0 or 1. 
# First we need to transform it to a 9x9 matrix with the right values 1,...,9.
soln_array = sum(i * soln[:,:,i] for i in 1:9)

## Presolving the Problem
Can you see the lines
```
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
```
? Cbc was able to use logic to deduce the value of every variable - no linear relaxation required! This "magic" is actually how a human might solve it. Consider the following:

We know that $x_{2,6,5}$ is fixed at 1 because it is one of the provided values. So we can actually replace $x_{2,6,5}$ wherever it appears in the constraints with the constant 1:

"The value 5 must appear in row 2":
$$x_{2,1,5} + x_{2,2,5} + x_{2,3,5} + x_{2,4,5} + x_{2,5,5} + x_{2,6,5} + x_{2,7,5} + x_{2,8,5} + x_{2,9,5} = 1$$
$$\rightarrow x_{2,1,5} + x_{2,2,5} + x_{2,3,5} + x_{2,4,5} + x_{2,5,5} + 1 + x_{2,7,5} + x_{2,8,5} + x_{2,9,5} = 1$$
$$\rightarrow x_{2,1,5} + x_{2,2,5} + x_{2,3,5} + x_{2,4,5} + x_{2,5,5} + x_{2,7,5} + x_{2,8,5} + x_{2,9,5} = 0$$

"The value 5 must appear in column 6":
$$x_{1,6,5} + x_{2,6,5} + x_{3,6,5} + x_{4,6,5} + x_{5,6,5} + x_{6,6,5} + x_{7,6,5} + x_{8,6,5} + x_{9,6,5} = 1$$
$$x_{1,6,5} + 1 + x_{3,6,5} + x_{4,6,5} + x_{5,6,5} + x_{6,6,5} + x_{7,6,5} + x_{8,6,5} + x_{9,6,5} = 1$$
$$x_{1,6,5} + x_{3,6,5} + x_{4,6,5} + x_{5,6,5} + x_{6,6,5} + x_{7,6,5} + x_{8,6,5} + x_{9,6,5} = 0$$

and so on. Because all those other variables can only be 0 or 1, and their sum is 0, they must all be 0. Thus Cbc presolve can perform the following procedure:
1. Replace all the fixed values with constants
2. Use constraints to fix variables, e.g. at 0 (or 1)
3. Go to 1 unless step 2 did nothing.

A small problem arises when there are multiple solutions to the problem - a random selection has to be made. Cbc will treat this case as "trying to find a feasible solution" - it will fix a variable, and follow the implications through.

Cbc can do similar presolving implications for continuous decisions too, e.g.
$$x \in \{ 0, 1 \}$$
$$x \leq \frac{1}{2}$$

will presolve to $x = 0$: Cbc knows that all the variables are integer, so we can safe round down the right-hand-side to the closest integer.

## Big $M$s and logical implications

We often need to model the following type of constraint:

> The constraint $a^Tx \leq b$ holds if $z=1$

An example of this appears in network problems:

> We have a network with a 'source' and a 'sink', and want to maximize flow between them across the network. We have a set of arcs we can build, at a cost, so our objective is to maximize profit from the flow less the cost of building these arcs.

If we say that $x_{i,j}$ is the flow from $i$ to $j$, and $z_{i,j}$ is a binary decision for whether we build the arc or not, we will have the constraint

$$ x_{i,j} > 0 \Longrightarrow z_{i,j}=1 $$

One way to express this using linear inequalities is to write

$$ 0 \leq x_{i,j} \leq M z_{i,j} $$

where $M$ is a **sufficiently large constant**. That means that $M$ is greater than the largest value $x_{i,j}$ would take in an optimal solution.

In some problems, picking a value for $M$ is easy - for example, if arcs have a maximum capacity $C$, set $M \leftarrow C$. In other cases, it is not clear *a priori* what the largest value $x_{i,j}$ can take is. It may be tempting to just choose a large value, like $M\leftarrow 10^{10}$, just to be safe. However this can make your MILP much harder to solve, as we will now demonstrate.

### Bad Big-Ms

Consider the following toy problem:

$$
\begin{align*}
\max\quad& x_1 + x_2 + x_3 \\
\text{s.t.}\quad& Ax \leq b \\
& 0 \leq x_i \leq M_i z_i \\
& \sum_{i=1}^3 z_i \leq 1 \\
& (x,z) \in \mathbb{R}^3 \times \{0,1\}^3
\end{align*}
$$

or, in words, pick the single largest $x_i$, subject to the linear constraints $Ax\leq b$. These linear constraints will imply some bounds on $x_i$, although it may be hard to find them. If we did know them, we could set $M_i$ to those bounds. Since we don't, we'll need to pick a "sufficiently large" $M$ from our understanding of the problem. Lets see what the solution of the linear relaxation of this problem looks like as we vary $M$.

In [ ]:
function example1(M)
    mod = Model(solver=CbcSolver(logLevel=0))
    @variable(mod, x[1:3] ≥ 0)
    @variable(mod, z[1:3], Bin)
    @objective(mod, Max, sum(x))
    
    # We will make our Ax≤b a very simple set!
    @constraint(mod, x[1] ≤ 4)
    @constraint(mod, x[2] ≤ 8)
    @constraint(mod, x[3] ≤ 9)

    @constraint(mod, bigm[i=1:3], x[i] ≤ M*z[i])
    @constraint(mod, sum(z) ≤ 1)
    
    solve(mod, relaxation=true)
    relax = getobjectivevalue(mod)
    
    solve(mod)
    opt = getobjectivevalue(mod)
    
    # MILP objective is 9, compare with LP objective
    gap = (relax-opt) / opt

    return 100gap, getvalue(x)[:], getvalue(z)[:]
end

using Plots

Mrange = 0:3:30
plot(Mrange, [example1(M)[1] for M in Mrange], xlabel="M", ylabel="Relaxation gap (%)", legend=false)
vline!([9])